In [1]:
import numpy as np
from class_lorenz96_mdl import LORENZ96
from class_state_vec import state_vector
from class_obs import obs_da
from class_da_sys import da_system
from sys import argv
import random as rd

#-----------------------------------------------------------------------
# Read the Lorenz-96 nature run
#-----------------------------------------------------------------------

infile = 'lorenz96_run.pkl'
sv = state_vector()
sv = sv.load(infile)
x_nature = sv.getTrajectory()
x_nature = x_nature[:,::5]
xdim,maxit = np.shape(x_nature)


#-----------------------------------------------------------------------
# Read the Lorenz-96 observations
#-----------------------------------------------------------------------
infile = 'y_obs.pkl'
obs = obs_da()
obs = obs.load(infile)

#-----------------------------------------------------------------------
# get the observed states
#-----------------------------------------------------------------------
yp = list(range(0,40)) #observe full states
#yp = list(np.arange(0,40,2)) #observe full states

if len(yp) < xdim:
  obs.reduceDim(yp)

y_obs = obs.getVal()
y_pts = obs.getPos()
y_err = obs.getErr()

ydim, ycolumn = np.shape(y_obs)

#-----------------------------------------------------------------------
# Initialize the da system
#-----------------------------------------------------------------------
das = da_system()
das.setStateVector(sv)
das.setObsData(obs)
das.xdim = xdim
das.ydim = ydim
das.x0 = x_nature[:,0]
das.t = sv.getTimes()
das.t0 = das.t[0]

#-----------------------------------------------------------------------
# Initialize the ensemble
#-----------------------------------------------------------------------
das.edim = 10 
das.ens_bias_init = 0
das.ens_sigma_init = 1

#-----------------------------------------------------------------------
# Initialize the error covariances B and R, and the linearized 
# observation operator H
#-----------------------------------------------------------------------

I = np.identity(xdim)

L = 5.47 #observe all states(optimal)
#L = 2.73
rad = 4
Localization_distance = int(3.65*L+1e-6)

I_r = np.identity(int(2*Localization_distance + 1))
print('shape I_r = ',np.shape(I_r))

for i in range(len(I_r)):
    for j in range(len(I_r)):
        if i != j:
            continue
        I_r[i,j] = np.exp(((abs(i-Localization_distance))**2)/(2*(5.5**2)))

print('I_r = ', I_r)

# Set background error covariance
sigma_b = 1.0
B = I * sigma_b**2

# Set observation error covariance

sigma_r = 1
R = I_r * sigma_r**2



# Set the linear observation operator matrix as I
H = I

const = 1.0
C = I * const

das.setB(B)
das.setR(R)
das.setH(H)
das.setC(C)

# Update the matrices to fit the reduced observation dimension
if len(yp) < xdim:
  das.reduceYdim(yp)

print('B = ')
print(das.getB())
print('R = ')
print(das.getR())
print('H = ')
print(das.getH())


#-----------------------------------------------------------------------
# Initialize the timesteps
#-----------------------------------------------------------------------
t_nature = sv.getTimes()

acyc_step = 1                       # (how frequently to perform an analysis)
dtau = (t_nature[acyc_step] - t_nature[0])
fcst_step = acyc_step                      # (may need to change for 4D DA methods)
fcst_dt = dtau / fcst_step

# Store basic timing info in das object
das.acyc_step = acyc_step
das.dtau = dtau
das.fcst_step = fcst_step
das.fcst_dt = fcst_dt
das.dt = (t_nature[1] - t_nature[0])
print('das.dt = ', das.dt)
das.maxit = maxit
das.xdim = xdim


#-----------------------------------------------------------------------
# Choose DA method:
#-----------------------------------------------------------------------

method='EnKF'


das.setMethod(method)

#-----------------------------------------------------------------------
# Store DA object
#-----------------------------------------------------------------------
name = 'x_analysis_init'
outfile=name+'.pkl'
das.save(outfile)

print(das)


shape I_r =  (39, 39)
I_r =  [[ 390.31033316    0.            0.         ...,    0.            0.            0.        ]
 [   0.          211.742709      0.         ...,    0.            0.            0.        ]
 [   0.            0.          118.73089006 ...,    0.            0.            0.        ]
 ..., 
 [   0.            0.            0.         ...,  118.73089006    0.            0.        ]
 [   0.            0.            0.         ...,    0.          211.742709
     0.        ]
 [   0.            0.            0.         ...,    0.            0.
   390.31033316]]
B = 
[[ 1.  0.  0. ...,  0.  0.  0.]
 [ 0.  1.  0. ...,  0.  0.  0.]
 [ 0.  0.  1. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  1.  0.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  0.  1.]]
R = 
[[ 390.31033316    0.            0.         ...,    0.            0.            0.        ]
 [   0.          211.742709      0.         ...,    0.            0.            0.        ]
 [   0.            0.  

In [3]:
3.65*5.47

19.9655

In [6]:
import numpy as np
list(np.arange(0,40,2))

20

In [26]:
np.shape(y_obs)

(40, 20)